In [ ]:
# GET PARAMETERS
import os 
import sys
import pickle

# Get Parent folder : 
current_path = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_path, '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)


# Personnal imports: 
from constants.paths import SAVE_DIRECTORY, FOLDER_PATH
from jupyter_ipynb.utils import parse_results_to_dataframe
from save.fast_training_str_results.subway_in_subway_out_bike_in_bike_out_1_trial_1year import results_string as results_string_1year
# from save.fast_training_str_results.subway_out_subway_in_bike_in_bike_out_1_trial_1year import results_string as results_string_1year
# from save.fast_training_str_results.subway_in_subway_out_bike_in_bike_out_1_trial_77days import results_string as results_string_77days
# ----
results_string = results_string_1year  # + results_string_1year_other_config# results_string_1year_other_config # results_string_77days #results_string_1year
# --- Load results 
results_df = parse_results_to_dataframe(results_string,bis=True)

results_df['contextuals'] = results_df.apply(lambda row : '_'.join([ds_name for ds_name in ['subway_in','subway_out','bike_in','bike_out','weather'] if row[ds_name]]),axis=1)
results_df['horizon'] = results_df['horizon'].apply(lambda h : int(h[1:-3]))
results_df['bis'] = results_df['bis'].astype(int)
results_df['id'] = results_df.apply(lambda row : f_name(row),axis=1)

results_df.fillna(False)
results_df[i]

/tmp/ipykernel_144533/2869016669.py:24: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  results_df.fillna(False)


,target_data,subway_in,subway_out,bike_in,bike_out,weather,epochs,horizon,horizon_code,bis,RMSE,MAE,MAPE,stack,ff_concat_late,attn_late,STAEformer,FFConcatLate,AttnLate,AttnKeepTempDim
0,subway_in,False,True,True,True,False,e500,+15min,h1,1,36.39,20.92,22.25,False,False,False,False,False,False,False
1,subway_in,False,True,False,True,False,e500,+15min,h1,1,36.76,20.94,21.34,False,False,False,False,False,False,False
2,subway_in,False,True,True,False,False,e500,+15min,h1,1,36.53,20.99,22.58,False,False,False,False,False,False,False
3,subway_in,False,True,False,False,False,e500,+15min,h1,1,36.87,21.05,22.84,False,False,False,False,False,False,False
4,subway_in,False,False,True,False,False,e500,+15min,h1,1,35.84,20.70,21.69,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,subway_in,False,False,False,False,False,e500,+60min,h4,5,44.97,24.08,22.95,False,False,False,True,False,False,False
231,subway_in,False,True,False,False,False,e500,+60min,h4,5,45.30,23.89,23.21,False,False,False,True,False,False,False
232,subway_in,False,True,False,False,False,e50,+60min,h4,5,44.71,24.19,24.70,False,False,False,True,False,False,False
233,subway_in,False,False,False,True,False,e100,+60min,h4,5,44.51,23.82,23.75,False,False,False,True,False,False,True


In [ ]:
def f_name(row):
    if row['STAEformer']:
        add_name = 'STAEformer'
    else:
        add_name = 'STGCN'
    if row['epochs'] != '':
        add_e = f"_{row['epochs']}"
    else:
        add_e = ''
    if row['stack']:
        return f"{add_name}_{row['contextuals']}_stack{add_e}_h{row['horizon']}"
    elif row['ff_concat_late']:
        return f"{add_name}_{row['contextuals']}_ff_concat_late{add_e}_h{row['horizon']}"
    elif row['attn_late']:
        return f"{add_name}_{row['contextuals']}_attn_late{add_e}_h{row['horizon']}" 
    else:
        return f"{add_name}_{row['contextuals']}{add_e}_h{row['horizon']}"
# ---


# --- Add MASE:
freq = 15 # 15min 


for target_data in ['subway_in'] : #['subway_in']: # ['subway_out']:
    for model_name in ['STGCN','STAEformer']:
        subfolder = f'K_fold_validation/training_wo_HP_tuning/optim/subway_in_{model_name}'
        path_model_args = f"{SAVE_DIRECTORY}/{subfolder}/best_models"
        model_args = pickle.load(open(f"{path_model_args}/model_args.pkl", 'rb'))
        for horizon in [1,2,3,4]:
            # for contextual_dataset_names in [['subway_in','bike_in','bike_out'],['subway_in','bike_out'],['subway_in','bike_in'],['bike_in'],[],['subway_in'],['bike_in','bike_out'],['bike_out']]: 
            for contextual_dataset_names in [['subway_out','bike_in','bike_out'],['subway_out','bike_out'],['subway_out','bike_in'],['bike_in'],['bike_out'],['bike_in','bike_out'],[],['subway_out'] ]: 
                for n_bis in range(1,6): # range(1,6):
                    for stack in [True,False]:
                        for ff_concat_late in [True,False]:
                            for attn_late in [True,False]:
                                for epochs in ['','e50','e500']: # ['',200]: # [100,200]:
                                    for weather in [True,False]: # ['',200]: # [100,200]:
                                        if model_name == 'STGCN':
                                            calendar_name = 'calendar_embedding'
                                        elif model_name == 'STAEformer':
                                            calendar_name = 'calendar'
                                        else:
                                            raise NotImplementedError
                                        dataset_names =  [target_data] +contextual_dataset_names+ [calendar_name]
                                        

                                        sub_df = results_df[(results_df['contextuals'] == '_'.join(contextual_dataset_names)) & 
                                                        (results_df['horizon'] == horizon*freq) &
                                                        (results_df['bis'] == n_bis)&
                                                        (results_df['stack'] == stack)&
                                                        (results_df['ff_concat_late'] == ff_concat_late) & 
                                                        (results_df['STAEformer'] == (model_name=='STAEformer'))&
                                                        (results_df['attn_late'] == attn_late)&
                                                        (results_df['epochs'] == epochs)&
                                                        (results_df['weather'] == weather)
                                                        ]
                                        if len(sub_df) == 0:
                                            continue
                                        idx = sub_df.index
                                        assert len(idx) == 1, f"Number of configuration recorded for {'_'.join(contextual_dataset_names)}: {len(idx)}. Has to be = 1.\n{sub_df}"

                                        if stack:
                                            name_i = f"{'_'.join(dataset_names)}_stack_h{horizon}_bis{n_bis}"
                                        elif ff_concat_late:
                                            name_i = f"{'_'.join(dataset_names)}_ff_concat_late_h{horizon}_bis{n_bis}"
                                        else:
                                            name_i = f"{'_'.join(dataset_names)}_h{horizon}_bis{n_bis}"
                                        if model_name == 'STAEformer':
                                            name_i = f"{name_i}"
                                        elif model_name == 'STGCN':
                                            name_i = f"_{name_i}"
                                        else:
                                            raise NotImplementedError
                                        results_df.loc[idx[0], 'MASE'] = model_args['model'][f"{name_i}_f5"]['performance']['test_metrics'][f'mase_h{horizon}']
                                        results_df.loc[idx[0], 'RMSE'] = model_args['model'][f"{name_i}_f5"]['performance']['test_metrics'][f'rmse_h{horizon}']
                                        results_df.loc[idx[0], 'MSE'] = model_args['model'][f"{name_i}_f5"]['performance']['test_metrics'][f'mse_h{horizon}']
                                        results_df.loc[idx[0], 'MAE'] = model_args['model'][f"{name_i}_f5"]['performance']['test_metrics'][f'mae_h{horizon}']
            # ---


    # --- Plot Metrics : 

from jupyter_ipynb.NetMob_training_analysis.plotting import plot_boxplot_on_metric 


plot_boxplot_on_metric(results_df,metric_i='MAE',xaxis_label = 'config', legend_group = 'horizon', save_path=None)
# plot_boxplot_on_metric(results_df,metric_i='RMSE',xaxis_label = 'config', legend_group = 'horizon', save_path=None)
# plot_boxplot_on_metric(results_df,metric_i='MASE',xaxis_label = 'config', legend_group = 'horizon', save_path=None)